# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
#Libraries
#Dataframe and arrays
import pandas as pd
import numpy as np


from scipy.spatial.distance import pdist, squareform

In [2]:
df = pd.read_excel('../data/online_fashion.xlsx')

In [3]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [4]:
df.shape

(541909, 8)

In [5]:
df.Quantity.unique()

array([     6,      8,      2,     32,      3,      4,     24,     12,
           48,     18,     20,     36,     80,     64,     10,    120,
           96,     23,      5,      1,     -1,     50,     40,    100,
          192,    432,    144,    288,    -12,    -24,     16,      9,
          128,     25,     30,     28,      7,     56,     72,    200,
          600,    480,     -6,     14,     -2,     11,     33,     13,
           -4,     -5,     -7,     -3,     70,    252,     60,    216,
          384,    -10,     27,     15,     22,     19,     17,     21,
           34,     47,    108,     52,  -9360,    -38,     75,    270,
           42,    240,     90,    320,   1824,    204,     69,    -36,
         -192,   -144,    160,   2880,   1400,     39,    -48,    -50,
           26,   1440,     31,     82,     78,     97,     98,     35,
           57,    -20,    110,    -22,    -30,    -70,   -130,    -80,
         -120,    -40,    -25,    -14,    -15,    -69,   -140,   -320,
      

In [6]:
df.sort_values(by='Quantity')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom
225529,556690,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom
225530,556691,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom
4287,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom
225528,556687,23003,Printing smudges/thrown away,-9058,2011-06-14 10:36:00,0.00,NaN,United Kingdom
115818,546152,72140F,throw away,-5368,2011-03-09 17:25:00,0.00,NaN,United Kingdom
431381,573596,79323W,"Unsaleable, destroyed.",-4830,2011-10-31 15:17:00,0.00,NaN,United Kingdom
341601,566768,16045,NaN,-3667,2011-09-14 17:53:00,0.00,NaN,United Kingdom
323458,565304,16259,NaN,-3167,2011-09-02 12:18:00,0.00,NaN,United Kingdom


In [7]:
df.isnull().sum() / df.shape[0] * 100.00

InvoiceNo       0.000000
StockCode       0.000000
Description     0.268311
Quantity        0.000000
InvoiceDate     0.000000
UnitPrice       0.000000
CustomerID     24.926694
Country         0.000000
dtype: float64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [9]:
df.InvoiceDate.min()

Timestamp('2010-12-01 08:26:00')

In [10]:
df.InvoiceDate.max()

Timestamp('2011-12-09 12:50:00')

## Decide what I want to drop

In [11]:
#Country = Unspecified 
#CustomerID
#Description

#####Price
#Unit price of 0 (zero)

#####Quantity
#Massive negative values

#####Items
#POSTAGE
#DOTCOM POSTAGE

In [12]:
df['Rev'] = df['Quantity']*df['UnitPrice']

In [13]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [14]:
#Top 10 Countries
df_top_countries_stg = df.groupby(['Country'])['Rev'].agg('sum')
df_top_countries = df_top_countries_stg.sort_values(ascending=False).head(10).to_frame()
df_top_countries

,Rev
Country,
United Kingdom,8.187806e+06
Netherlands,2.846615e+05
EIRE,2.632768e+05
Germany,2.216982e+05
France,1.974039e+05
Australia,1.370773e+05
Switzerland,5.638535e+04
Spain,5.477458e+04
Belgium,4.091096e+04


In [15]:
df_top_countries = df_top_countries.reset_index()


In [16]:
top_country_rev = []
for i in df_top_countries['Country'][:3]:
    top_country_rev.append(i)
top_country_rev    

['United Kingdom', 'Netherlands', 'EIRE']

In [17]:
#Top 10 Description by Revenue - All Countries
df_top_items_rev = df.groupby(['Description'])['Rev'].agg('sum')
df_top_items_rev.sort_values(ascending=False).head(10)

Description
DOTCOM POSTAGE                        206245.48
REGENCY CAKESTAND 3 TIER              164762.19
WHITE HANGING HEART T-LIGHT HOLDER     99668.47
PARTY BUNTING                          98302.98
JUMBO BAG RED RETROSPOT                92356.03
RABBIT NIGHT LIGHT                     66756.59
POSTAGE                                66230.64
PAPER CHAIN KIT 50'S CHRISTMAS         63791.94
ASSORTED COLOUR BIRD ORNAMENT          58959.73
CHILLI LIGHTS                          53768.06
Name: Rev, dtype: float64

In [18]:
#Top 10 Description by Quantity - All Countries
df_top_items_vol = df.groupby(['Description'])['Quantity'].agg('sum')
df_top_items_vol.sort_values(ascending=False).head(10)

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     53847
JUMBO BAG RED RETROSPOT               47363
ASSORTED COLOUR BIRD ORNAMENT         36381
POPCORN HOLDER                        36334
PACK OF 72 RETROSPOT CAKE CASES       36039
WHITE HANGING HEART T-LIGHT HOLDER    35317
RABBIT NIGHT LIGHT                    30680
MINI PAINT SET VINTAGE                26437
PACK OF 12 LONDON TISSUES             26315
PACK OF 60 PINK PAISLEY CAKE CASES    24753
Name: Quantity, dtype: int64

In [19]:



for i in top_country_rev:
    print(df.loc[df['Country'] == i].groupby(['Description'])['Quantity'].agg('sum').sort_values(ascending=False).head(3))
    

Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS    48326
JUMBO BAG RED RETROSPOT              43167
POPCORN HOLDER                       34365
Name: Quantity, dtype: int64
Description
RABBIT NIGHT LIGHT      4801
SPACEBOY LUNCH BOX      4528
DOLLY GIRL LUNCH BOX    4132
Name: Quantity, dtype: int64
Description
PACK OF 72 RETROSPOT CAKE CASES    1728
60 TEATIME FAIRY CAKE CASES        1536
VINTAGE SNAP CARDS                 1492
Name: Quantity, dtype: int64


In [20]:
df_days_week = df

In [21]:
df_days_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
Rev            541909 non-null float64
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 37.2+ MB


In [22]:
df_days_week['Weekday'] = df_days_week['InvoiceDate'].dt.day_name()
df_days_week.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Rev,Weekday
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday


In [23]:
#customers with more than one country
df_cust_country = df.groupby(['CustomerID','Country']).count()
#df_cust_country.to_frame
df_cust_country

,,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Rev,Weekday
CustomerID,Country,,,,,,,,
12346.0,United Kingdom,2,2,2,2,2,2,2,2
12347.0,Iceland,182,182,182,182,182,182,182,182
12348.0,Finland,31,31,31,31,31,31,31,31
12349.0,Italy,73,73,73,73,73,73,73,73
12350.0,Norway,17,17,17,17,17,17,17,17
12352.0,Norway,95,95,95,95,95,95,95,95
12353.0,Bahrain,4,4,4,4,4,4,4,4
12354.0,Spain,58,58,58,58,58,58,58,58
12355.0,Bahrain,13,13,13,13,13,13,13,13


In [24]:
#Revenue by Day of Week - All Countries
df_day_of_week_rev = df_days_week.groupby(['Weekday'])['Rev'].agg('sum')
df_day_of_week_rev.sort_values(ascending=False)#.head(10)

Weekday
Thursday     2112519.000
Tuesday      1966182.791
Wednesday    1734147.010
Monday       1588609.431
Friday       1540610.811
Sunday        805678.891
Name: Rev, dtype: float64

In [25]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Rev,Weekday
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,Wednesday
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,Wednesday
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,Wednesday


In [26]:
#Customers with Negative Qty
#Identify Customer/ Product mix that are negative - i.e. potential returns from outside reporting period

df_cust_neg_qty = df.groupby(['CustomerID', 'StockCode'])['Quantity'].agg('sum').to_frame()
df_cust_neg_qty.reset_index(inplace=True)
df_cust_neg_qty = df_cust_neg_qty.loc[(df_cust_neg_qty['Quantity'] <= 0)]
df_cust_neg_qty

df_new = df[~(df['StockCode'].isin(df_cust_neg_qty.StockCode) & df['CustomerID'].isin(df_cust_neg_qty.CustomerID))]

df_new.shape

(434567, 10)

In [27]:
#Find & Remove Strange StockCodes
df_unique_stock_codes = df['StockCode'].unique()
df_unique_stock_codes = pd.DataFrame(df_unique_stock_codes)
df_unique_stock_codes['len'] = df_unique_stock_codes[0].astype(str).str.len()
df_unique_stock_codes.rename(columns={0:'StockCode'}, inplace=True)
df_unique_stock_codes.sort_values(by='len')
df_unique_stock_codes['len'] = df_unique_stock_codes['len'].astype(int)
#df_unique_stock_codes.loc[(df_unique_stock_codes['len'] >= 8) | (df_unique_stock_codes['len'] <= 4)].sort_values(by='len')

df_stock_to_drop_stg = df_unique_stock_codes.loc[(df_unique_stock_codes['len'] >= 8) | (df_unique_stock_codes['len'] <= 4)].sort_values(by='len')
df_stock_to_drop_stg.reset_index(drop=True)
df_stock_to_drop = pd.concat([df_stock_to_drop_stg.iloc[0:10], df_stock_to_drop_stg.iloc[-6:], df_stock_to_drop_stg.iloc[-9:-8] ])
df_stock_to_drop

df_new = df_new[~(df['StockCode'].isin(df_stock_to_drop.StockCode) )]

df_new.shape


C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


(432457, 10)

In [28]:
#Remove rows where the CustomerID is Nan
df_new.dropna(subset=['CustomerID'], inplace=True)


In [29]:
#Stratisfying Customers based on Spend During Reporting Period

# PART 1
rev_by_cust = df_new.groupby("CustomerID")["Rev"].sum().to_frame()
rev_by_cust

rev_by_cust['quantile'] = pd.qcut(rev_by_cust['Rev'], 100, labels=False)
rev_by_cust

# Emanuele's way didnt work. 
conditions = [
    (rev_by_cust['quantile'] >= 75 ) & (rev_by_cust['quantile'] <= 95),
    (rev_by_cust['quantile'] > 95)]
choices = ['Preferred', 'VIP']
rev_by_cust['Status'] = np.select(conditions, choices, default='Regular')
print(rev_by_cust)


                Rev  quantile     Status
CustomerID                              
12347.0     4310.00        95  Preferred
12348.0     1437.24        79  Preferred
12349.0     1457.55        79  Preferred
12350.0      294.40        28    Regular
12352.0      245.13        24    Regular
12353.0       89.00         4    Regular
12354.0     1079.40        72    Regular
12355.0      459.40        45    Regular
12356.0     2487.43        89  Preferred
12357.0     6207.67        97        VIP
12358.0      928.06        67    Regular
12359.0     1446.13        79  Preferred
12360.0     2302.06        88  Preferred
12361.0      174.90        15    Regular
12362.0     1679.43        82  Preferred
12363.0      552.00        50    Regular
12364.0     1208.10        75  Preferred
12365.0      140.34        11    Regular
12367.0      150.90        12    Regular
12370.0     3425.69        93  Preferred
12371.0     1527.96        80  Preferred
12372.0     1196.04        74    Regular
12373.0      324

In [30]:
#Top Returning Customers
df_top_returners = df.groupby(['CustomerID'])['Quantity'].agg('sum').to_frame()
df_top_returners.reset_index(inplace=True)
df_top_returners = df_top_returners.loc[(df_top_returners['Quantity'] < 0)].sort_values(by='Quantity', ascending=True)
df_top_returners

,CustomerID,Quantity
3103,16546.0,-303
2578,15823.0,-283
1384,14213.0,-244
3245,16742.0,-189
2892,16252.0,-158
3650,17307.0,-144
3829,17548.0,-132
4350,18256.0,-70
264,12666.0,-56
2436,15638.0,-52


In [31]:
#Min Max Mode Median for Each StockCode
#HELP!!!!
df_product_price_variance = df.groupby(['StockCode'])['UnitPrice'].agg('min')
df_product_price_variance

StockCode
10002            0.00
10080            0.00
10120            0.21
10125            0.42
10133            0.00
10134            0.00
10135            0.25
11001            0.83
15030            0.00
15034            0.07
15036            0.00
15039            0.53
16008            0.00
16010            0.00
16011            0.21
16012            0.21
16014            0.32
16015            0.00
16016            0.72
16033            0.00
16043            0.00
16045            0.00
16046            0.00
16048            0.12
16049            0.00
16052            0.00
16053            0.00
16054            0.12
16216            0.06
16218            0.06
                ...  
C2               0.00
CRUK             1.60
D                0.01
DCGS0003         0.00
DCGS0004        16.63
DCGS0055         0.00
DCGS0057         0.00
DCGS0066P        0.00
DCGS0067         0.00
DCGS0068         0.00
DCGS0069         0.00
DCGS0070         0.00
DCGS0071         0.00
DCGS0072         0.00


In [32]:
#Top Invoice Values by Rev
df_top_invoices_rev = df.groupby(['InvoiceNo', 'CustomerID'])['Rev'].agg('sum').to_frame()
df_top_invoices_rev.reset_index(inplace=True)
df_top_invoices_rev = df_top_invoices_rev.sort_values(by='Rev', ascending=False)
df_top_invoices_rev

,InvoiceNo,CustomerID,Rev
18503,581483,16446.0,168469.60
1909,541431,12346.0,77183.60
7926,556444,15098.0,38970.00
12419,567423,17450.0,31698.16
8112,556917,12415.0,22775.93
14459,572209,18102.0,22206.00
12411,567381,17450.0,22104.80
10843,563614,12415.0,21880.44
5463,550461,15749.0,21535.90
14371,572035,14646.0,20277.92


In [33]:
#Top Invoice Values by Qty
df_top_invoices_qty = df.groupby(['InvoiceNo', 'CustomerID'])['Quantity'].agg('sum').to_frame()
df_top_invoices_qty.reset_index(inplace=True)
df_top_invoices_qty = df_top_invoices_qty.sort_values(by='Quantity', ascending=False)
df_top_invoices_qty

,InvoiceNo,CustomerID,Quantity
18503,581483,16446.0,80995
1909,541431,12346.0,74215
8112,556917,12415.0,15049
10612,563076,14646.0,14730
14371,572035,14646.0,13392
12419,567423,17450.0,12572
17360,578841,13256.0,12540
6437,552883,14646.0,12266
10843,563614,12415.0,12196
10344,562439,12931.0,11848


In [34]:
#Top Customer by Rev
df_top_customer_rev = df.groupby(['CustomerID'])['Rev'].agg('sum').to_frame()
df_top_customer_rev.reset_index(inplace=True)
df_top_customer_rev = df_top_customer_rev.sort_values(by='Rev', ascending=False)
df_top_customer_rev['CustomerID'] = df_top_customer_rev['CustomerID'].astype(int)
df_top_customer_rev

,CustomerID,Rev
1703,14646,279489.02
4233,18102,256438.49
3758,17450,187482.17
1895,14911,132572.62
55,12415,123725.45
1345,14156,113384.14
3801,17511,88125.38
3202,16684,65892.08
1005,13694,62653.10
2192,15311,59419.34


In [35]:
#Find Strange Descriptions
df_unique_descriptions = df['Description'].unique()
df_unique_descriptions = pd.DataFrame(df_unique_descriptions)
df_unique_descriptions['len'] = df_unique_descriptions[0].astype(str).str.len()
df_unique_descriptions.rename(columns={0:'Description'}, inplace=True)
df_unique_descriptions.sort_values(by='len')
df_unique_descriptions['len'] = df_unique_descriptions['len'].astype(int)
df_unique_descriptions.loc[(df_unique_descriptions['len'] >= 36) | (df_unique_descriptions['len'] <= 11)].sort_values(by='len', ascending=False)

#put into consistent case i.e. upper or lower

,Description,len
1615,FROG CANDLE,11
3998,michel oops,11
4139,damages wax,11
2975,thrown away,11
3779,CORDIAL JUG,11
3292,wrong code?,11
4193,????missing,11
4013,stock check,11
1631,SKULLS TAPE,11
4177,dotcomstock,11


In [36]:
#Find descriptions with negative works like 'damaged' 'fees' 'mouldy', 'faulty', 'wet', 'lost', 'missing','wrong', 'broke'

In [37]:
#Get word frequency to assist with categorisation
#errors out as some descriptons are rubbish
#search lookup string to find counterpart part II: qwerty1wd

from collections import defaultdict

import pandas as pd

text_list = []

#need to get unique descriptions

df_uniq_descs = df['Description'].unique()

for i in df_uniq_descs[:395]:
    text_list.append(i)

word_freq = defaultdict(int)

for text in text_list:
    for word in text.split():
        word_freq[word] += 1

pd.DataFrame.from_dict(word_freq, orient='index') \
.sort_values(0, ascending=False) \
.rename(columns={0: 'abs_freq'})
    
    
    

,abs_freq
RED,36
SET,36
OF,33
HEART,27
RETROSPOT,26
BAG,26
BOX,24
PINK,23
METAL,17
WHITE,16


In [38]:
#same as above - this works too!
#wrod count
#search lookup string to find counterpart part I: qwerty1wd
text_list = []

#need to get unique descriptions

word_freq = defaultdict(int)

for text in df['Description'][:395].unique():
    for word in text.split():
        word_freq[word] += 1

pd.DataFrame.from_dict(word_freq, orient='index') \
.sort_values(0, ascending=False) \
.rename(columns={0: 'abs_freq'})
    

,abs_freq
RED,26
SET,25
OF,23
HEART,20
RETROSPOT,20
PINK,19
BAG,16
BOX,16
WHITE,15
VINTAGE,14


In [39]:
len(df['StockCode'].unique())

4070

In [40]:
len(df['CustomerID'].unique())

4373

In [41]:
#Finding Customers with more than 1 Country listed
df_customer_stg = df.groupby('CustomerID')['Country'].unique()
df_customer_stg.loc[df_customer_stg.apply(lambda x:len(x)>1)]

CustomerID
12370.0           [Cyprus, Austria]
12394.0          [Belgium, Denmark]
12417.0            [Belgium, Spain]
12422.0    [Australia, Switzerland]
12429.0          [Denmark, Austria]
12431.0        [Australia, Belgium]
12455.0             [Cyprus, Spain]
12457.0       [Switzerland, Cyprus]
Name: Country, dtype: object

In [42]:
pd.DataFrame(df_customer_stg)

,Country
CustomerID,
12346.0,[United Kingdom]
12347.0,[Iceland]
12348.0,[Finland]
12349.0,[Italy]
12350.0,[Norway]
12352.0,[Norway]
12353.0,[Bahrain]
12354.0,[Spain]
12355.0,[Bahrain]


In [43]:
#Finding StockCodes with  more than 1 Description listed
# NOTE THAT I USED THE NEW DATAFRAME HERE
df_inventory_stg = df_new.groupby('StockCode')['Description'].unique()
df_inventory_stg.loc[df_inventory_stg.apply(lambda x:len(x)>1)]

#greater than 1 = 191
#greater than 2 = 16
#greater than 3 = 2

StockCode
20622             [VIPPASSPORT COVER , VIP PASSPORT COVER ]
21109     [LARGE CAKE TOWEL, CHOCOLATE SPOTS, LARGE CAKE...
21112     [SWISS ROLL TOWEL, PINK  SPOTS, SWISS ROLL TOW...
21175     [GIN + TONIC DIET METAL SIGN, GIN AND TONIC DI...
21232     [STRAWBERRY CERAMIC TRINKET BOX, STRAWBERRY CE...
21243         [PINK  POLKADOT PLATE , PINK POLKADOT PLATE ]
21507     [ELEPHANT, BIRTHDAY CARD, , ELEPHANT BIRTHDAY ...
21811     [CHRISTMAS HANGING HEART WITH BELL, HANGING HE...
21818     [GLITTER CHRISTMAS HEART , GLITTER HEART DECOR...
21899             [KEY FOB , GARAGE DESIGN, GARAGE KEY FOB]
21928     [JUMBO BAG SCANDINAVIAN PAISLEY, JUMBO BAG SCA...
22129     [PARTY CONES CANDY TREE DECORATION, PARTY CONE...
22134     [LADLE LOVE HEART RED , MINI LADLE LOVE HEART ...
22135     [LADLE LOVE HEART PINK, MINI LADLE LOVE HEART ...
22179     [SET 10 LIGHTS NIGHT OWL, SET 10 NIGHT OWL LIG...
22197                [SMALL POPCORN HOLDER, POPCORN HOLDER]
22268     [EASTER DECORATION S

In [44]:
#df_new['UnitPrice'].loc[(df_new['StockCode'] == 22776)].unique()
df_new.loc[(df_new['StockCode'] == 23126)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Rev,Weekday
107430,545445,23126,DOLLCRAFT GIRL AMELIE KIT,4,2011-03-02 16:26:00,4.95,17738.0,United Kingdom,19.80,Wednesday
113147,545914,23126,DOLLCRAFT GIRL AMELIE KIT,4,2011-03-08 11:39:00,4.95,13340.0,United Kingdom,19.80,Tuesday
113794,545998,23126,FELTCRAFT GIRL AMELIE KIT,4,2011-03-08 14:35:00,4.95,13373.0,United Kingdom,19.80,Tuesday
113900,546000,23126,DOLLCRAFT GIRL AMELIE,4,2011-03-08 14:47:00,4.95,13089.0,United Kingdom,19.80,Tuesday
114675,546037,23126,FELTCRAFT GIRL AMELIE KIT,4,2011-03-09 10:00:00,4.95,14825.0,United Kingdom,19.80,Wednesday
115437,546120,23126,DOLLCRAFT GIRL AMELIE,4,2011-03-09 14:23:00,4.95,16945.0,United Kingdom,19.80,Wednesday
115551,546133,23126,DOLLCRAFT GIRL AMELIE KIT,4,2011-03-09 15:20:00,4.95,14291.0,United Kingdom,19.80,Wednesday
119745,546569,23126,FELTCRAFT GIRL AMELIE KIT,4,2011-03-15 10:53:00,4.95,14016.0,EIRE,19.80,Tuesday
126340,547071,23126,FELTCRAFT GIRL AMELIE KIT,4,2011-03-20 13:59:00,4.95,12693.0,Germany,19.80,Sunday
130312,547485,23126,FELTCRAFT GIRL AMELIE KIT,2,2011-03-23 11:16:00,4.95,17019.0,United Kingdom,9.90,Wednesday


In [45]:
#df_new['CustomerID'].isna().sum()

In [46]:
#df_new.dropna(subset=['CustomerID'],inplace=True)


In [47]:
#Calculate Weighted Average Price (AWP) Part I
## NOTE USED DF_NEW
df_awp_trial = df_new[['StockCode','Quantity','Rev']]
df_awp_trial.groupby('StockCode').agg({'Quantity': ['sum'], 'Rev': ['sum']})
##need to divide SUM by QTY to give AWP


,Quantity,Rev
,sum,sum
StockCode,,
10002,823,699.55
10080,291,114.41
10120,193,40.53
10125,1226,930.30
10133,1839,876.87
10135,1937,1785.44
11001,458,774.02
15030,143,41.47


In [48]:
#Calculate Weighted Average Price (AWP) PArt II
#df_AWP_stg = df.groupby(['StockCode'])['Quantity','UnitPrice'].agg('sum',count)
#df_AWP_stg


df.groupby('StockCode').agg({'Quantity': ['sum','count'], 'UnitPrice': ['sum','count']})

# sum(qty*unit_price - by row) / sum(total Qty)


#df_AWP = df_AWP_stg[lambda x: x<=0]
#df_AWP = pd.DataFrame(df_AWP)
#df_AWP

Quantity         UnitPrice      
                  sum count         sum count
StockCode                                    
10002            1037    73      77.150    73
10080             495    24       9.040    24
10120             193    30       6.300    30
10125            1296    94      80.810    94
10133            2775   200     129.160   200
10134             -19     1       0.000     1
10135            2230   180     253.830   180
11001            1430   120     225.380   120
15030             293    14       3.770    14
15034            5206   142      52.030   142
15036           22552   524     557.230   524
15039            2065   149     183.530   149
16008            1382    71       9.530    71
16010              23     4       0.360     4
16011            1773    47       9.870    47
16012            1244    37       7.770    37
16014           13328    54      21.680    54
16015             373    27      11.720    27
16016            1273    65      54.990    65
16033             960     5       0.480     5
16043              53     3       0.240     3
16045            4333    71       3.000    71
16046              89    18      14.450    18
16048            1461    69      22.430    69
16049             547     7       1.200     7
16052             463    14       7.660    14
16053            -102     1       0.000     1
16054            1058    34       4.380    34
16216            3333    45       6.860    45
16218            3821    64      17.270    64
...               ...   ...         ...   ...
C2                290   144    7098.000   144
CRUK              -16    16    7933.430    16
D               -1194    77    5581.310    77
DCGS0003           -3     5       9.940     5
DCGS0004            1     1      16.630     1
DCGS0055           -1     1       0.000     1
DCGS0057           -6     1       0.000     1
DCGS0066P          -3     1       0.000     1
DCGS0067          -11     1       0.000     1
DCGS0068          -10     1       0.000     1
DCGS0069           -4     2      15.790     2
DCGS0070           -7     2      12.720     2
DCGS0071           -2     1       0.000     1
DCGS0072           -1     1       0.000     1
DCGS0073           -4     1       0.000     1
DCGS0074           -1     1       0.000     1
DCGS0076            3     2      32.260     2
DCGSSBOY           47    11      34.150    11
DCGSSGIRL          47    13      40.730    13
DOT              1707   710  206252.060   710
M                3164   571  214448.410   571
PADS                4     4       0.003     4
POST             6353  1256   46388.470  1256
S                 -59    63    3175.490    63
gift_0001_10       39     9      66.640     9
gift_0001_20       20    10     150.380    10
gift_0001_30       37     8     175.530     8
gift_0001_40        3     3     100.700     3
gift_0001_50        4     4     167.560     4
m                   1     1       2.550     1

[4070 rows x 4 columns]

#Calculate Weighted Average Price (AWP) PArt III
grouped = df.groupby('Date')

def wavg(group):
    d = group['value']
    w = group['wt']
    return (d * w).sum() / w.sum()

grouped.apply(wavg)

In [49]:
#Find Min, Max, Mean of StockCodes
df_item_prices = df[['StockCode', 'UnitPrice']]
df_item_prices.groupby(by='StockCode').agg([min, max, 'mean'])

UnitPrice                       
                   min        max        mean
StockCode                                    
10002             0.00      1.660    1.056849
10080             0.00      0.850    0.376667
10120             0.21      0.210    0.210000
10125             0.42      1.660    0.859681
10133             0.00      1.660    0.645800
10134             0.00      0.000    0.000000
10135             0.25      2.510    1.410167
11001             0.83      3.360    1.878167
15030             0.00      0.290    0.269286
15034             0.07      0.850    0.366408
15036             0.00      1.630    1.063416
15039             0.53      1.660    1.231745
16008             0.00      0.250    0.134225
16010             0.00      0.120    0.090000
16011             0.21      0.210    0.210000
16012             0.21      0.210    0.210000
16014             0.32      0.420    0.401481
16015             0.00      0.500    0.434074
16016             0.72      0.850    0.846000
16033             0.00      0.120    0.096000
16043             0.00      0.120    0.080000
16045             0.00      0.120    0.042254
16046             0.00      0.850    0.802778
16048             0.12      0.850    0.325072
16049             0.00      0.420    0.171429
16052             0.00      0.840    0.547143
16053             0.00      0.000    0.000000
16054             0.12      0.420    0.128824
16216             0.06      1.250    0.152444
16218             0.06      2.130    0.269844
...                ...        ...         ...
C2                0.00    150.000   49.291667
CRUK              1.60   1100.440  495.839375
D                 0.01   1867.860   72.484545
DCGS0003          0.00      2.510    1.988000
DCGS0004         16.63     16.630   16.630000
DCGS0055          0.00      0.000    0.000000
DCGS0057          0.00      0.000    0.000000
DCGS0066P         0.00      0.000    0.000000
DCGS0067          0.00      0.000    0.000000
DCGS0068          0.00      0.000    0.000000
DCGS0069          0.00     15.790    7.895000
DCGS0070          0.00     12.720    6.360000
DCGS0071          0.00      0.000    0.000000
DCGS0072          0.00      0.000    0.000000
DCGS0073          0.00      0.000    0.000000
DCGS0074          0.00      0.000    0.000000
DCGS0076         16.13     16.130   16.130000
DCGSSBOY          1.25      3.290    3.104545
DCGSSGIRL         1.25      3.290    3.133077
DOT               0.00   4505.170  290.495859
M                 0.00  38970.000  375.566392
PADS              0.00      0.001    0.000750
POST              0.00   8142.750   36.933495
S                 2.80    570.000   50.404603
gift_0001_10      0.00      8.330    7.404444
gift_0001_20      0.00     17.020   15.038000
gift_0001_30      0.00     25.530   21.941250
gift_0001_40     33.33     34.040   33.566667
gift_0001_50     41.67     42.550   41.890000
m                 2.55      2.550    2.550000

[4070 rows x 3 columns]

In [50]:
df_new.to_csv('../data/cleaned_df2.csv')

def tagforremoval_negqty(row):
    '''
    Input: Row (which will be slicked into columns)
    Output: 1 or 0 depening on if row is to be deleted
    
    The purpose of this function is to tag rows for deletion.
    '''
    if row['CustomerID'].isin(df_cust_neg_qty.CustomerID) & row['StockCode'].isin(df_cust_neg_qty.StockCode):
        return 1
    else:
        return 0
